In [239]:
import numpy as np

In [282]:
# this should get 0.33
y = np.asarray([[0,1,0],[1,0,0],[0,0,1],[0,0,1]]).astype("float32")
py = np.asarray([[0,1,0],[1,0,0],[1,0,0],[1,0,0]]).astype("float32")

In [288]:
# this should get 1.0
y = np.asarray([[0,1,0],[1,0,0],[0,0,1],[0,0,1]]).astype("float32")
py = np.asarray([[0,1,0],[1,0,0],[0,0,1],[0,0,1]]).astype("float32")

In [294]:
# should be 0.273
y = np.asarray([[0,0,1],[1,0,0],[0,0,1],[0,0,1]]).astype("float32")
py = np.asarray([[0,1,0],[1,0,0],[1,0,0],[0,0,1]]).astype("float32")

In [295]:
y

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]], dtype=float32)

In [296]:
py

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]], dtype=float32)

In [300]:
po = np.dot(y.T, py)
print po
po = po / np.sum(po)
print po

[[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  1.]]
[[ 0.25  0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.25  0.25  0.25]]


In [301]:
np.dot(po.sum(axis=1), po.sum(axis=0))

0.3125

In [302]:
(np.trace(po) - np.dot(po.sum(axis=1), po.sum(axis=0))) / (1 - np.dot(po.sum(axis=1), po.sum(axis=0)))

0.27272727272727271

In [273]:
np.dot(y.T, -np.log(py+0.00001))

array([[ -1.00135303e-05,   1.15129251e+01,   1.15129251e+01],
       [  1.15129251e+01,  -1.00135303e-05,   1.15129251e+01],
       [ -2.00270606e-05,   2.30258503e+01,   2.30258503e+01]], dtype=float32)

In [281]:
-np.log(py+0.000001)

array([[  1.38155107e+01,  -9.53673862e-07,   1.38155107e+01],
       [ -9.53673862e-07,   1.38155107e+01,   1.38155107e+01],
       [ -9.53673862e-07,   1.38155107e+01,   1.38155107e+01],
       [ -9.53673862e-07,   1.38155107e+01,   1.38155107e+01]], dtype=float32)

----------

In [122]:
def random_one_hot(n):
    tot = []
    for i in range(n):
        arr = [0,0,0]
        arr[ np.random.randint(0,3) ] = 1
        tot.append(arr)
    return np.asarray(tot, dtype="float32")

In [360]:
random_one_hot(10)

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]], dtype=float32)

In [342]:
w = np.ones((3,3))
w[0,0]=0
w[1,1]=0
w[2,2]=0
w

array([[ 0.,  1.,  1.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.]])

In [343]:
numerator = np.dot(y.T, py)
print numerator

[[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  1.]]


In [344]:
denominator = np.outer(y.sum(axis=0), py.sum(axis=0))
print denominator

[[ 2.  1.  1.]
 [ 0.  0.  0.]
 [ 6.  3.  3.]]


In [338]:
denominator = denominator / np.sum(numerator)

In [339]:
numerator*w

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.01,  0.01,  0.  ]])

In [340]:
denominator*w

array([[ 0.   ,  0.025,  0.025],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.15 ,  0.075,  0.   ]])

In [341]:
1 - (np.sum(numerator*w) / np.sum(denominator*w))

0.92727272618900647

-----------

In [29]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.updates import *
import numpy as np

In [109]:
l_in = InputLayer((None, 3))
#l_dense = DenseLayer(l_in, num_units=3, nonlinearity=softmax)
#l_out = l_dense
X = T.fmatrix('X')
py = get_output(l_in, X)

In [110]:
y = T.fmatrix('y')

In [111]:
w = np.ones((3,3))
for i in range(0, 3):
    for j in range(0, 3):
        w[i,j] = (i-j)**2
w

array([[ 0.,  1.,  4.],
       [ 1.,  0.,  1.],
       [ 4.,  1.,  0.]])

In [228]:
def qwk(predictions, targets):
    w = np.ones((3,3))
    for i in range(0, 3):
        for j in range(0, 3):
            w[i,j] = (i-j)**2
    numerator = T.dot(targets.T, predictions)
    denominator = T.outer(targets.sum(axis=0), predictions.sum(axis=0))
    denominator = denominator / T.sum(numerator)  
    qwk = (T.sum(numerator*w) / T.sum(denominator*w))
    return qwk

In [229]:
def test(predictions, targets):
    w = np.ones((3,3))
    for i in range(0, 3):
        for j in range(0, 3):
            w[i,j] = (i-j)**2
    numerator = T.dot(targets.T, predictions)
    denominator = T.outer(targets.sum(axis=0), predictions.sum(axis=0))
    denominator = denominator / T.sum(numerator)  
    return numerator, denominator

In [218]:
def kp(predictions, targets):
    w = np.ones((3,3))
    w[0][0]=0
    w[1][1]=0
    w[2][2]=0
    numerator = T.dot(targets.T, predictions)
    denominator = T.outer(targets.sum(axis=0), predictions.sum(axis=0))
    denominator = denominator / T.sum(numerator)
    qwk = (T.sum(numerator*w) / T.sum(denominator*w))
    return qwk

In [220]:
loss = qwk(py, y)

In [83]:
params = get_all_params(l_out, trainable=True)

In [84]:
updates = nesterov_momentum(loss, params, 0.01, 0.9)

In [85]:
updates

OrderedDict([(W, Elemwise{add,no_inplace}.0),
             (b, Elemwise{add,no_inplace}.0),
             (<TensorType(float64, matrix)>, Elemwise{sub,no_inplace}.0),
             (<TensorType(float64, vector)>, Elemwise{sub,no_inplace}.0)])

In [86]:
train_fn = theano.function([X, y], loss, updates=updates)

In [87]:
y_train = np.asarray([[0,1,0],[1,0,0],[1,0,0],[0,0,1]]).astype("float32")
y_train

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]], dtype=float32)

In [88]:
X_train = np.random.normal(0,1,size=y_train.shape).astype("float32")
X_train

array([[ 1.85365927,  0.65476787,  0.05348646],
       [ 0.791933  ,  1.89965367,  1.27587533],
       [ 0.70612597, -0.21445501, -0.26872301],
       [ 0.74303192,  0.0707793 , -0.74300873]], dtype=float32)

In [106]:
train_fn(X_train, y_train)

array(0.9873062590789431)

In [115]:
c1 = np.asarray([[0,1,0],[1,0,0],[0,0,1],[0,0,1]]).astype("float32")
c2 = np.asarray([[0,1,0],[1,0,0],[1,0,0],[1,0,0]]).astype("float32")

In [121]:
y1 = T.fmatrix('y1')
y2 = T.fmatrix('y2')
1 - (kp(y1,y2).eval({y1:c1, y2:c2}))

0.33333333333333337

---------

In [173]:
def random_prob_dist(n):
    mat = np.abs(np.random.normal(0,1,size=(n,3)))
    for i in range(n):
        mat[i,] = mat[i,] / np.sum(mat[i,])
    return mat.astype("float32")

In [212]:
qwk(y1,y2).eval({y1:random_one_hot(100), y2:random_one_hot(100)})

Sum{acc_dtype=float64}.0
Sum{acc_dtype=float64}.0


array(0.7757897890592064)

In [231]:
r1 = random_one_hot(10)
r2 = random_one_hot(10)

In [238]:
num, den = test(y1,y2)
print sum(sum(num.eval({y1:r1, y2:r2})))
print sum(sum(den.eval({y1:r1, y2:r2})))

10.0
10.0


In [186]:
np.argmax(random_prob_dist(10),axis=1)

array([1, 0, 0, 1, 0, 0, 1, 2, 1, 1])

In [208]:
random_one_hot(10).sum(axis=0)

array([ 4.,  3.,  3.], dtype=float32)

In [307]:
-np.log(0.001)

6.9077552789821368

In [314]:
from lasagne.objectives import *

In [315]:
x1 = T.tensor4('x1')
x2 = T.tensor4('x2')


In [319]:
binary_crossentropy(x1,x2).eval({x1:np.zeros((10,5,20,20))+0.001, x2:np.zeros((10,5,20,20))})

array([[[[ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         ..., 
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005]],

        [[ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           0.0010005],
         ..., 
         [ 0.0010005,  0.0010005,  0.0010005, ...,  0.0010005,  0.0010005,
           

In [321]:
?binary_crossentropy

In [324]:
tmp = InputLayer((None, 16, 768, 1152))
for i in range(6):
    tmp = MaxPool2DLayer(tmp, pool_size=5, stride=2)
tmp.output_shape

(None, 16, 9, 15)